In [13]:
import json 

In [14]:
finegrained_baseline_path = '/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/finegrained_annotations.json'

In [15]:
with open(finegrained_baseline_path, 'r') as fp:
    data = json.load(fp)


In [16]:
data.keys()

dict_keys(['1-5', '13-1', '13829296', '13821251', '14-10', '6-1', '24-8', '13828403', '13828739', '0-7', '13727920', '15-1', '10-2', '10-5', '13728157', '13716964', '13863152', '13820610-1', '13731446', '19-2', '13821554', '13681560', '16-4', '13681084', '30-1', '16-2', '3-4', '13814886', '13729445', '34-5', '27-4', '13682111', '13829728', '0-8', '13731495', '23-0', '15-3', '12-1', '13829183', '18-4', '3-5', '13829773', '10-4', '13829919', '14-0', '13731140-1', '0-1', '13680457', '13681924', '31-5', '0-2', '13728101', '13814235', '14-1', '13820513', '24-6', '13862773', '13828227', '20-2', '28-1', '13682100', '0-10', '13828297', '13680391', '29-1', '13682475', '13809941', '13680171', '29-4', '13829710', '13-11', '0-5', '13811786', '27-1', '9-2', '34-2', '13863098', '13731361', '32-5', '8-4', '13716087', '4-1', '13828334', '1-1', '13612149-1', '12-5', '13716405', '13829607', '20-7', '28-5', '13729106-1', '13-4', '13681755', '13680968', '13811007', '13821588', '33-2', '2-3', '11-5', '20-4

In [20]:
data['13821251']

{'utterances': ['Sally: Hi guys have you heard about Natalie?',
  'Tom: No. What about her?',
  "Jane: You're such a gossip girl, Sally!",
  "Sally: Come on! I'm sure you're all curious.",
  'Tom: I am.',
  "Sally: Relax Jane, what's your problem?!",
  "Jane: She told us in confidence, I don't think we should share it with others.",
  "Greg: Now I'm also curious",
  'Henriette: Tell us!',
  'Henriette: Is she pregnant?',
  'Sally: 💣💣💣',
  "Sally: Guess who's the father!!!",
  'Henriette: I knew it!',
  'Henriette: Is it Mike?',
  'Sally: Dave!!!',
  'Henriette: WHAT????',
  "Greg: That's insane!"],
 'query': 'n/a',
 'target_summary': "Natalie is pregnant with Dave. Jane didn't think Sally should share it with others as Natalie told them that in confidence. Henriette and Greg are surprised that the father is Dave not Mike.",
 'dataset_name': 'SAMSum',
 'group_idx': 1,
 'annotations': {'gpt3_finetune': {'model_prediction': 'Natalie had a one-night stand with Dave and she is not pregnant.

In [9]:

df_dict = {'Dialogue': [], 
           'Model': [], 
           'Summary': [], 
           'origin': [],
           'Annotations':[],
           'Reference_Summary': []
          }
for docid, doc_ann in data.items():
    utterance = '\n'.join(doc_ann['utterances'])
    reference_summary = doc_ann['target_summary']
    dataset_name = doc_ann['dataset_name']
    annotations = doc_ann['annotations']
    if doc_ann['query'] == 'n/a':
        for model_name, model_annotations in annotations.items():
            df_dict['Dialogue'].append(utterance)
            df_dict['Model'].append(model_name)
            df_dict['Summary'].append(model_annotations['model_prediction'])
            df_dict['Reference_Summary'].append(reference_summary)
            df_dict['origin'].append(dataset_name)
            sentences_dict = {}
            for sentence_annotations in model_annotations['sentence_annotations']:
                for error_annotation in sentence_annotations['annotation']:
                        err_span = error_annotation['error_span']
                        err_type = error_annotation['intrinsic/extrinsic']
                        err_class = error_annotation['error_class']
                        if err_span.strip():
                            if sentence_annotations['sentence'] not in sentences_dict:
                                sentences_dict[sentence_annotations['sentence']] = []
                                sentences_dict[sentence_annotations['sentence']] += [(err_span, err_type, err_class)]
            df_dict['Annotations'].append(sentences_dict)
                
    


In [10]:
import pandas as pd
df_finegrained = pd.DataFrame(df_dict)

In [11]:
df_finegrained.to_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_finegrained_aggrefact.csv')

In [12]:
df_finegrained

,Dialogue,Model,Summary,origin,Annotations,Reference_Summary
0,"Natacha: hi, i can come and pick you up at the...",gpt3_finetune,Charles will probably arrive at the train stat...,SAMSum,{'Charles will probably arrive at the train st...,Charles has just landed and he will be at RER ...
1,"Natacha: hi, i can come and pick you up at the...",pegasus,Natacha will pick Charles up at the RER at 5:3...,SAMSum,{'Natacha will pick Charles up at the RER at 5...,Charles has just landed and he will be at RER ...
2,"Natacha: hi, i can come and pick you up at the...",structure_aware_bart,Natacha will pick Charles up at the RER at 5:3...,SAMSum,{'Natacha will pick Charles up at the RER at 5...,Charles has just landed and he will be at RER ...
3,"Natacha: hi, i can come and pick you up at the...",condigsum,Natacha will pick Charles up at the RER statio...,SAMSum,{'Natacha will pick Charles up at the RER stat...,Charles has just landed and he will be at RER ...
4,"Natacha: hi, i can come and pick you up at the...",bart,Natacha will pick Charles up at the station Ve...,SAMSum,{'Natacha will pick Charles up at the station ...,Charles has just landed and he will be at RER ...
...,...,...,...,...,...,...
290,"Mark: So, we've got our where and when. Packag...",gpt3_finetune,"Mark, Anna, George and Julia are going on a pa...",SAMSum,{},They are going to do some research on holiday ...
291,"Mark: So, we've got our where and when. Packag...",condigsum,"Mark, Julia, Anna, George and George are going...",SAMSum,"{'Mark, Julia, Anna, George and George are goi...",They are going to do some research on holiday ...
292,"Mark: So, we've got our where and when. Packag...",bart,"Mark, Anna, Julia, George and Anna are going o...",SAMSum,"{'Mark, Anna, Julia, George and Anna are going...",They are going to do some research on holiday ...
293,"Mark: So, we've got our where and when. Packag...",pegasus,"Mark, Anna, George and Julia are looking for a...",SAMSum,{},They are going to do some research on holiday ...
